In [5]:
import pandas as pd
import re
from mlxtend.frequent_patterns import apriori, association_rules
data = pd.read_csv("Team_Sports_Survey_Preprocessed.csv")
# remove the eg. for each items
def remove_parentheses(text):
    if isinstance(text, str):  
        return re.sub(r"\s*\(.*?\)", "", text)
    return text  

In [6]:
# Clean the columns data by removing the (eg. items) by using the regex
columns_to_combine = ["Preferred Swimming Pool Activities", "Preferred Beach Activities", "Preferred Spa and Wellness Activities"]
data[columns_to_combine] = data[columns_to_combine].applymap(remove_parentheses)

# Combine all the columns
data["Combined Activities"] = data[columns_to_combine].apply(lambda x: ', '.join(x.dropna()), axis=1)

# Convert the items into a transactional data 
transactions = data["Combined Activities"].str.split(",").explode().str.strip().reset_index()
transactions_encoded = transactions.pivot_table(index="index", columns="Combined Activities", aggfunc=lambda x: 1, fill_value=0)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_6604\507130932.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[columns_to_combine] = data[columns_to_combine].applymap(remove_parentheses)


In [7]:
# Use Apriori algorithm to find frequent association
frequent_items= apriori(transactions_encoded, min_support=0.4, use_colnames=True)# threshold 40%
rule = association_rules(frequent_items)
rule[['antecedents','consequents','support', 'confidence', 'lift']].sort_values("confidence", ascending=False)

C:\Users\joaqu\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift
0,(Beach Swimming),(Relaxing in Pool),0.58,0.865672,1.109835
2,(Massage Therapy),(Relaxing in Pool),0.41,0.820000,1.051282
1,(Kayaking/Paddling),(Relaxing in Pool),0.40,0.816327,1.046572
